In [4]:
import os
import pandas as pd
directory = "raw"

In [36]:
df = pd.read_stata(os.path.join(directory, 'interview__actions.dta'))
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

df['interview_actions_idx'] = df.groupby('interview__id').transform('first').index

# we assume the interview__actions file to be sorted by survey solutions, if not add row below
# df.sort_values(['interview__id', 'datetime'], inplace=True)

# get the index of the first reject/review event for each interview
df_rejected = df[df['action'].isin(['RejectedBySupervisor', 'RejectedByHQ', 'OpenedBySupervisor', 'OpenedByHQ'])]
first_reject_idx = df_rejected.groupby('interview__id')['datetime'].idxmin()

# get dataframe where each row is the last 'Completed' event prior to first reject event for each interview
def get_last_complete(group):
    reject_time = first_reject_idx.get(group.name)
    if reject_time is not None:
        group = group.loc[group.index < reject_time]
    if any(group['action'] == 'Completed'):
        return group[group['action'] == 'Completed'].iloc[[-1]]
    else:
        return pd.DataFrame()

last_complete_df = df.groupby('interview__id').apply(get_last_complete)

# create a new dataframe with interviewer and datetime
interview_df = last_complete_df[['originator', 'datetime', 'responsible__name', 'interview_actions_idx']].copy()
interview_df.columns = ['interviewer', 'datetime', 'supervisor', 'interview_actions_idx']

# Add column for the total number of interviews per interviewer
interview_df['total_interviews'] = interview_df.groupby('interviewer')['interviewer'].transform('count')

# Add column for the sequential number of each interview per interviewer
interview_df.sort_values(['interviewer', 'datetime'], inplace=True)
interview_df['interview_sequence'] = interview_df.groupby('interviewer').cumcount() + 1

# Add column for the date of each interview
interview_df['date'] = interview_df['datetime'].dt.date

# Calculate the number of days passed since the first date
interview_df['days_since_start'] = (interview_df['datetime'] - interview_df.groupby('interviewer')['datetime'].transform('min')).dt.days


# bring in columns from the diagnostics file
diagnostics_df = pd.read_stata(os.path.join(directory, 'interview__diagnostics.dta'))
interview_df = interview_df.merge(diagnostics_df, on='interview__id', how='outer')  # or left join, if we want to filter out those who do not have a completed event


#interview_df = interview_df.sort_values(by='interview_actions_idx')
interview_df

,interview__id,interviewer,datetime,supervisor,interview_actions_idx,total_interviews,interview_sequence,date,days_since_start,interview__key,interview__status,responsible,interviewers,rejections__sup,rejections__hq,entities__errors,questions__comments,interview__duration,n_questions_unanswered
0,5cb9ddfc494844b7bf97930c613be160,AbdelmajM_int_N,2023-03-09 14:48:40,MoezL_sup_N,25.0,4.0,1.0,2023-03-09,0.0,47-95-93-34,ApprovedBySupervisor,AbdelmajM_int_N,1,0,0,0,0,00.00:10:09,0
1,bba844ccc4f542118079c9411bef68e5,AbdelmajM_int_N,2023-03-15 12:56:58,MoezL_sup_N,166.0,4.0,2.0,2023-03-15,5.0,54-72-27-96,ApprovedBySupervisor,AbdelmajM_int_N,1,0,0,0,0,00.01:11:31,0
2,4e3f0194aa8d4b8da1b61d25373ad270,AbdelmajM_int_N,2023-03-15 19:58:31,MoezL_sup_N,135.0,4.0,3.0,2023-03-15,6.0,22-49-76-44,ApprovedBySupervisor,AbdelmajM_int_N,1,0,0,0,0,00.00:49:03,0
3,4ca870c778f648cc986dfdafa83e3755,AbdelmajM_int_N,2023-04-12 08:24:32,MoezL_sup_N,188.0,4.0,4.0,2023-04-12,33.0,63-89-53-12,ApprovedBySupervisor,AbdelmajM_int_N,1,0,0,0,0,00.03:23:17,0
4,6c0de8d6f11646f69e89a0bf64c57810,HoudaR_int_N,2023-03-13 18:24:22,HoudaR_sup_N,81.0,3.0,1.0,2023-03-13,0.0,30-90-89-62,ApprovedBySupervisor,HoudaR_int_N,1,0,0,0,0,00.00:46:54,0
5,0d75e9acc3d24dffba9313c8e6fac8ed,HoudaR_int_N,2023-03-28 21:48:38,HoudaR_sup_N,123.0,3.0,2.0,2023-03-28,15.0,09-53-02-51,ApprovedBySupervisor,HoudaR_int_N,1,0,0,0,0,00.00:28:40,0
6,3896dfd4d24143ab9c18eeb0f082392d,HoudaR_int_N,2023-04-13 08:58:10,HoudaR_sup_N,178.0,3.0,3.0,2023-04-13,30.0,00-13-03-79,ApprovedBySupervisor,HoudaR_int_N,1,0,0,0,0,00.00:21:16,0
7,f7fe612b614644529b673ecc59e4d18b,HoudaZ_int_N,2023-03-08 19:43:25,HoudaZ_sup_N,41.0,6.0,1.0,2023-03-08,0.0,33-02-53-81,ApprovedBySupervisor,HoudaZ_int_N,1,1,0,0,0,00.00:10:57,0
8,f5d0050536e74bb59c746556398f6e25,HoudaZ_int_N,2023-04-03 19:34:17,HoudaZ_sup_N,234.0,6.0,2.0,2023-04-03,25.0,24-57-01-56,ApprovedBySupervisor,HoudaZ_int_N,1,0,0,0,0,00.01:37:35,0
9,f199b3df198b486b95f4f8ffd9a7090e,HoudaZ_int_N,2023-04-07 10:23:39,HoudaZ_sup_N,220.0,6.0,3.0,2023-04-07,29.0,18-64-00-44,ApprovedBySupervisor,HoudaZ_int_N,1,0,0,0,0,00.00:23:50,0


# variables
interviewer: interviewer who first completed the interviewer (note: may be reassigned to other interviewer afterwards, but rare)
datetime: the datetime of the first completion (note: may be rejected and again marked as complete, several times)
supervisor: the supervisor assigned to interviewer at the time (note: field organisation may differ)
interview_actions_idx: index of the row on the interview_actions file from which information was picked
total_interview: total interviews done by the interviewer
interview_sequence: sequential number by interviewer of the interview, by first completion date
date: date of first completion
days_since_start: first completion date in days since first completion in the data set
rest of variables is from interview_diagnostics

